# Titanic Survival Prediction

Dataset Description :   

<br>PassengerId: and id given to each traveler on the boat
<br>Pclass: the passenger class. It has three possible values: 1,2,3 (first, second and third class)
<br>The Name of the passeger
<br>The Sex
<br>The Age
<br>SibSp: number of siblings and spouses traveling with the passenger
<br>Parch: number of parents and children traveling with the passenger
<br>The ticket number
<br>The ticket Fare
<br>The cabin number
<br>The embarkation. This describes three possible ports at which people boarded Titanic. Three possible values S,C,Q

In [1]:
train_location = 'train.csv'
Original_train_location = 'trainOriginal.csv'
test_location = 'test.csv'
Original_test_location = 'testOriginal.csv'

In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
import matplotlib as plt

C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
trainData = pd.read_csv(train_location)
testData = pd.read_csv(test_location)

In [4]:
trainData.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,0,22.0,1,0,7.2500,1.0,0
1,1,1,38.0,1,0,71.2833,0.0,1
2,3,1,26.0,0,0,7.9250,1.0,1
3,1,1,35.0,1,0,53.1000,1.0,1
4,3,0,35.0,0,0,8.0500,1.0,0


In [5]:
trainData.shape

(891, 8)

In [6]:
testData.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,1
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,1
4,3,1,22.0,1,1,12.2875,1


In [7]:
testData.shape

(418, 7)

## Replacing Name with title 

Adding column titles:
    <br> 0 - Mr.
    <br> 1 - Master.
    <br> 2 - Mrs.
    <br> 3 - Miss.
    

In [8]:
titles = []
k=0
for i in trainData.iloc[:,0]:
    foundTitle = 0
    for title in i.split():
        if (title in ['Mr.', 'Don.', 'Dr.', 'Major.', 'Col.', 'Rev.', 'Sir.', 'Capt.']):
            titles.append(0)
            foundTitle = 1
            break
        elif (title == 'Master.'):
            titles.append(1)
            foundTitle = 1
            break
        elif (title in ['Mrs.', 'Mme.', 'Lady.', 'Countess.']):
            titles.append(2)
            foundTitle = 1
            break
        elif (title in ['Miss.', 'Ms.', 'Mlle.']):
            titles.append(3)
            foundTitle = 1
            break
    
    if(foundTitle == 0):
        titles.append(0)
        print(i)

AttributeError: 'numpy.int64' object has no attribute 'split'

In [9]:
len(titles)

0

In [10]:
trainData.insert(loc=1, column='Title', value=titles)
trainData = trainData.drop(labels=['Name'], axis=1) 
trainData.head()

ValueError: Length of values does not match length of index

In [11]:
titles = []
k=0
for i in testData.iloc[:,0]:
    foundTitle = 0
    for title in i.split():
        if (title in ['Mr.', 'Don.', 'Dr.', 'Major.', 'Col.', 'Rev.', 'Sir.', 'Capt.']):
            titles.append(0)
            foundTitle = 1
            break
        elif (title == 'Master.'):
            titles.append(1)
            foundTitle = 1
            break
        elif (title in ['Mrs.', 'Mme.', 'Lady.', 'Countess.']):
            titles.append(2)
            foundTitle = 1
            break
        elif (title in ['Miss.', 'Ms.', 'Mlle.']):
            titles.append(3)
            foundTitle = 1
            break
    
    if(foundTitle == 0):
        titles.append(3)
        print(i)

AttributeError: 'numpy.int64' object has no attribute 'split'

In [12]:
len(titles)

0

In [13]:
testData.insert(loc=1, column='Title', value=titles)
testData = testData.drop(labels=['Name'], axis=1) 
testData.head()

ValueError: Length of values does not match length of index

## Loading train and test data 
<br> With age - (predicting age with help Title and Fare) 
<br> Without age (dropping those rows where age is missing)

In [14]:
trainData.iloc[:,[0,1,2,3,6,8]].head()

IndexError: positional indexers are out-of-bounds

In [15]:
trainData = trainData.iloc[:,[0,1,2,3,6,8]]

IndexError: positional indexers are out-of-bounds

In [16]:
X_train_WA = trainData.iloc[:,:5].values
Y_train_WA = trainData.iloc[:,5].values

In [17]:
# dropping any row where age is nan because it is a huge factor
trainData_WOA = trainData.dropna(how='any',axis=0) 
X_train_WOA = trainData_WOA.iloc[:,:5].values
Y_train_WOA = trainData_WOA.iloc[:,5].values

In [18]:
testData.shape

(418, 7)

In [19]:
testOriginalData = pd.read_csv(Original_test_location)

In [20]:
X_test_WA = testData.iloc[:,:8].values

## Replacing missing Age with the mean of respective Titles (Mr/Master/Mrs/Miss)

In [21]:
ageMr = 0
countMr = 0
ageMaster = 0
countMaster = 0
ageMrs = 0
countMrs = 0
ageMiss = 0
countMiss = 0
missing = 0

for index, row in trainData.iterrows():
    if(row.isnull().any()):
        missing = missing + 1
    else:
        if(row['Title'] == 0):
            ageMr = ageMr + row['Age']
            countMr = countMr + 1 
        elif(row['Title'] == 1):
            ageMaster = ageMaster + row['Age']
            countMaster = countMaster + 1
        elif(row['Title'] == 2):
            ageMrs = ageMrs + row['Age']
            countMrs = countMrs + 1
        elif(row['Title'] == 3):
            ageMiss = ageMiss + row['Age']
            countMiss = countMiss + 1   

KeyError: 'Title'

In [22]:
# Counting number of Mr, Master, Mrs, Miss, Missing Age in training set
print(countMr, countMaster, countMrs, countMiss, missing)

418 36 111 149 177


In [23]:
# Their average Ages
print(ageMr/countMr, ageMaster/countMaster, ageMrs/countMrs, ageMiss/countMiss)

33.02272727272727 4.574166666666667 35.873873873873876 21.845637583892618


In [24]:
# Replacing Missing Age values
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        if(row['Title'] == 0):
            trainData.set_value(index,'Age',ageMr/countMr)
        elif(row['Title'] == 1):
            trainData.set_value(index,'Age',ageMaster/countMaster)
        elif(row['Title'] == 2):
            trainData.set_value(index,'Age',ageMrs/countMrs)
        elif(row['Title'] == 3):
            trainData.set_value(index,'Age',ageMiss/countMiss)

In [25]:
trainData.head(10)

,Title,Pclass,Sex,Age,Fare,Survived
0,0,3,0,22.000000,7.2500,0
1,2,1,1,38.000000,71.2833,1
2,3,3,1,26.000000,7.9250,1
3,2,1,1,35.000000,53.1000,1
4,0,3,0,35.000000,8.0500,0
5,0,3,0,33.022727,8.4583,0
6,0,1,0,54.000000,51.8625,0
7,1,3,0,2.000000,21.0750,0
8,2,3,1,27.000000,11.1333,1
9,2,2,1,14.000000,30.0708,1


In [26]:
# Replacing the 2 missing embarked values with 0 as frequency of embarked = 0 is highest
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        trainData.set_value(index,'Embarked',0)

In [27]:
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        print('Missing value found')

For Test set

In [28]:
for index, row in testData.iterrows():
    if(row.isnull().any()):
        if(row['Title'] == 0):
            testData.set_value(index,'Age',ageMr/countMr)
        elif(row['Title'] == 1):
            testData.set_value(index,'Age',ageMaster/countMaster)
        elif(row['Title'] == 2):
            testData.set_value(index,'Age',ageMrs/countMrs)
        elif(row['Title'] == 3):
            testData.set_value(index,'Age',ageMiss/countMiss)

In [29]:
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        print('Missing value found')

All missing values have been dealt with

In [30]:
X_train_WA = trainData.iloc[:,:5].values
Y_train_WA = trainData.iloc[:,5].values
X_test_WA = testData.iloc[:,:5].values

## With Scaling

In [31]:
# scaling 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
X_train_WA = scaler.fit_transform(X_train_WA)
X_test_WA = scaler.fit_transform(X_test_WA)

In [32]:
X_train_WA.shape

(891, 5)

In [33]:
# One Hot encoding of Y_train
from keras.utils import to_categorical
Y_train_WA = to_categorical(Y_train_WA)
Y_train_WA.shape

C:\Users\DELL PC\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(891, 2)

In [34]:
X_test_WA.shape

(418, 5)

# ANN


In [35]:
import keras
from keras import layers
from keras import models
from keras import optimizers

In [36]:
model = models.Sequential()

In [37]:
model.add(layers.Dense(16, input_shape=(5,), activation='relu'))
# the model now takes input of dimensions (*,5)
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(2, activation='softmax'))

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                96        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 32)                544       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 66        
Total params: 978
Trainable params: 978
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
X_train_WA.shape

(891, 5)

In [41]:
model.fit(x=X_train_WA, y=Y_train_WA, epochs=200, batch_size=None, verbose=2, validation_split=0.25, shuffle=True)

Train on 668 samples, validate on 223 samples
Epoch 1/200
 - 1s - loss: 0.7152 - acc: 0.4281 - val_loss: 0.6834 - val_acc: 0.4215
Epoch 2/200
 - 0s - loss: 0.6598 - acc: 0.6647 - val_loss: 0.6426 - val_acc: 0.7892
Epoch 3/200
 - 0s - loss: 0.6348 - acc: 0.7455 - val_loss: 0.6075 - val_acc: 0.7892
Epoch 4/200
 - 0s - loss: 0.5979 - acc: 0.7754 - val_loss: 0.5669 - val_acc: 0.7892
Epoch 5/200
 - 0s - loss: 0.5596 - acc: 0.7844 - val_loss: 0.5200 - val_acc: 0.7892
Epoch 6/200
 - 0s - loss: 0.5197 - acc: 0.7859 - val_loss: 0.4807 - val_acc: 0.7892
Epoch 7/200
 - 0s - loss: 0.4969 - acc: 0.7919 - val_loss: 0.4602 - val_acc: 0.7803
Epoch 8/200
 - 0s - loss: 0.4961 - acc: 0.7799 - val_loss: 0.4550 - val_acc: 0.7848
Epoch 9/200
 - 0s - loss: 0.4900 - acc: 0.7859 - val_loss: 0.4505 - val_acc: 0.7848
Epoch 10/200
 - 0s - loss: 0.4843 - acc: 0.7889 - val_loss: 0.4463 - val_acc: 0.7803
Epoch 11/200
 - 0s - loss: 0.4657 - acc: 0.7844 - val_loss: 0.4430 - val_acc: 0.7803
Epoch 12/200
 - 0s - loss: 0

Epoch 97/200
 - 0s - loss: 0.4390 - acc: 0.8024 - val_loss: 0.3994 - val_acc: 0.8296
Epoch 98/200
 - 0s - loss: 0.4436 - acc: 0.8039 - val_loss: 0.3972 - val_acc: 0.8117
Epoch 99/200
 - 0s - loss: 0.4420 - acc: 0.8219 - val_loss: 0.3971 - val_acc: 0.8117
Epoch 100/200
 - 0s - loss: 0.4436 - acc: 0.7949 - val_loss: 0.3968 - val_acc: 0.8251
Epoch 101/200
 - 0s - loss: 0.4348 - acc: 0.8024 - val_loss: 0.3952 - val_acc: 0.8161
Epoch 102/200
 - 0s - loss: 0.4523 - acc: 0.8039 - val_loss: 0.3962 - val_acc: 0.8251
Epoch 103/200
 - 0s - loss: 0.4415 - acc: 0.7949 - val_loss: 0.3964 - val_acc: 0.8161
Epoch 104/200
 - 0s - loss: 0.4450 - acc: 0.8024 - val_loss: 0.3972 - val_acc: 0.8206
Epoch 105/200
 - 0s - loss: 0.4436 - acc: 0.8009 - val_loss: 0.3953 - val_acc: 0.8072
Epoch 106/200
 - 0s - loss: 0.4359 - acc: 0.8069 - val_loss: 0.3958 - val_acc: 0.8251
Epoch 107/200
 - 0s - loss: 0.4320 - acc: 0.8054 - val_loss: 0.3953 - val_acc: 0.8251
Epoch 108/200
 - 0s - loss: 0.4409 - acc: 0.7919 - val_lo

Epoch 193/200
 - 0s - loss: 0.4309 - acc: 0.8174 - val_loss: 0.3893 - val_acc: 0.8386
Epoch 194/200
 - 0s - loss: 0.4272 - acc: 0.8129 - val_loss: 0.3870 - val_acc: 0.8430
Epoch 195/200
 - 0s - loss: 0.4303 - acc: 0.8054 - val_loss: 0.3852 - val_acc: 0.8386
Epoch 196/200
 - 0s - loss: 0.4264 - acc: 0.8069 - val_loss: 0.3854 - val_acc: 0.8430
Epoch 197/200
 - 0s - loss: 0.4267 - acc: 0.8039 - val_loss: 0.3856 - val_acc: 0.8386
Epoch 198/200
 - 0s - loss: 0.4296 - acc: 0.7949 - val_loss: 0.3875 - val_acc: 0.8430
Epoch 199/200
 - 0s - loss: 0.4280 - acc: 0.8144 - val_loss: 0.3844 - val_acc: 0.8341
Epoch 200/200
 - 0s - loss: 0.4294 - acc: 0.8204 - val_loss: 0.3851 - val_acc: 0.8475


In [42]:
predictions = model.predict_classes(X_test_WA)
predictions.shape

(418,)

In [43]:
testDataOriginal = pd.read_csv(Original_test_location)
testDataOriginal["PassengerId"];

In [44]:
results = pd.DataFrame({
    "PassengerId": testDataOriginal["PassengerId"],
    "Survived": predictions})

In [45]:
results.to_csv('titanic_ANN6.csv', index=False)